https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/gan.py
https://stackoverflow.com/questions/40994583/how-to-implement-tensorflows-next-batch-for-own-data

In [2]:
import xml.etree.ElementTree as ET
import csv


def get_labels_dict(data_path):
    labels_dict = {}
    with open(data_path + 'sci_labels.csv', 'r') as f:
        file = csv.reader(f)
        for row in file:
            labels_dict[row[0]] = row[1]
    return labels_dict


def get_features_labels(root, labels_dict):
    corpus = [] # each row is a string formed from all messages in a conversations
    labels = [] # each row is 0 or 1, corresponds to label for same row in corpus

    for conversation in root:
        string = " "
        for message in conversation:
            text = message.find('text').text
            if text is not None:
                string = string + "\r\n" + text 
        corpus.append(string)
        labels.append(int(labels_dict[conversation.get('id')]))
    return corpus, labels

In [3]:
train_data_path = '../../data/svm_training_data/'
training_xml = ET.parse(train_data_path + 'training_data.xml')
train_root = training_xml.getroot()

test_data_path = '../../data/svm_test_data/'
test_data_src = '../../data/pan12-sexual-predator-identification-test-corpus-2012-05-21/'
test_xml = ET.parse(test_data_src + 'pan12-sexual-predator-identification-test-corpus-2012-05-17.xml')
test_root = test_xml.getroot()

train_corpus, train_labels = get_features_labels(train_root, get_labels_dict(train_data_path))
test_corpus, test_labels = get_features_labels(test_root, get_labels_dict(test_data_path))

train_corpus_norm = []
train_corpus_susp = []
train_labels_norm = []
train_labels_susp = []
for index in range(len(train_corpus)):
    if train_labels[index] == 1:
        train_corpus_susp.append(train_corpus[index])
        train_labels_susp.append(train_labels[index])
    else:
        train_corpus_norm.append(train_corpus[index])
        train_labels_norm.append(train_labels[index])

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MaxAbsScaler
import scipy
# from sklearn.model_selection import train_test_split
import numpy as np

vectorizer = TfidfVectorizer()
_ = vectorizer.fit_transform(train_corpus)
X_train_norm = vectorizer.transform(train_corpus_norm)
X_train_susp = vectorizer.transform(train_corpus_susp)
X_test = vectorizer.transform(test_corpus)

X_train_norm = scipy.sparse.csr_matrix(X_train_norm, dtype=np.float32)
y_train_norm = np.array(train_labels_norm)
X_train_susp = scipy.sparse.csr_matrix(X_train_susp, dtype=np.float32)
y_train_susp = np.array(train_labels_susp)
X_test = scipy.sparse.csr_matrix(X_test, dtype=np.float32)
y_test = np.array(test_labels)

print(np.min(X_train_norm[:][0]))
print(np.max(X_train_norm[:][0]))

scaler = MaxAbsScaler()
X_train_norm = scaler.fit_transform(X_train_norm)
X_train_susp = scaler.fit_transform(X_train_susp)
X_test = scaler.fit_transform(X_test)
X_train_norm.data -= 0.5
X_train_susp.data -= 0.5
X_test.data -= 0.5
X_train_norm.data *= 2
X_train_susp.data *= 2
X_test.data *= 2

print(np.min(X_train_norm[:][0]))
print(np.max(X_train_norm[:][0]))

# X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=87)
# print("Train data shape:{}\r\nTest data shape:{}".format(X_train.shape, X_test.shape))

0.0
0.44709805
-0.96498847
1.0


In [5]:
# print(X_train_norm[:][0])
print(np.mean(X_train_norm[:][0]))

-0.0010184544


In [6]:
""" Generative Adversarial Networks (GAN).
Using generative adversarial networks (GAN) to generate digit images from a
noise distribution.
References:
    - Generative adversarial nets. I Goodfellow, J Pouget-Abadie, M Mirza,
    B Xu, D Warde-Farley, S Ozair, Y. Bengio. Advances in neural information
    processing systems, 2672-2680.
    - Understanding the difficulty of training deep feedforward neural networks.
    X Glorot, Y Bengio. Aistats 9, 249-256
Links:
    - [GAN Paper](https://arxiv.org/pdf/1406.2661.pdf).
    - [MNIST Dataset](http://yann.lecun.com/exdb/mnist/).
    - [Xavier Glorot Init](www.cs.cmu.edu/~bhiksha/courses/deeplearning/Fall.../AISTATS2010_Glorot.pdf).
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import division, print_function, absolute_import

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Import MNIST data
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Training Params
num_steps = 10
batch_size = 100 #128
learning_rate = 0.0002

# Network Params
image_dim = X_train_norm.shape[1] #784 # 28*28 pixels
gen_hidden_dim = 10 #256
disc_hidden_dim = 10 #256
noise_dim = X_train_norm.shape[1] # 100 # Noise data points
print(X_train_norm.shape[1])

# A custom initialization (see Xavier Glorot init)
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

# Store layers weight & bias
weights = {
    'gen_hidden1': tf.Variable(glorot_init([noise_dim, gen_hidden_dim])),
    'gen_out': tf.Variable(glorot_init([gen_hidden_dim, image_dim])),
    'disc1_hidden1': tf.Variable(glorot_init([image_dim, disc_hidden_dim])),
    'disc1_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
    'disc2_hidden1': tf.Variable(glorot_init([image_dim, disc_hidden_dim])),
    'disc2_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
}
biases = {
    'gen_hidden1': tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_out': tf.Variable(tf.zeros([image_dim])),
    'disc1_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc1_out': tf.Variable(tf.zeros([1])),
    'disc2_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc2_out': tf.Variable(tf.zeros([1])),
}


# Generator
def generator(x):
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['gen_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['gen_out'])
    out_layer = tf.add(out_layer, biases['gen_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer


# Discriminator
def discriminator_SCI(x): # is D in paper
    hidden_layer = tf.matmul(x, weights['disc1_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc1_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['disc1_out'])
    out_layer = tf.add(out_layer, biases['disc1_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

def discriminator_gvr(x): # is D prime in paper, discriminator_generated_vs_real
    hidden_layer = tf.matmul(x, weights['disc2_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc2_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['disc2_out'])
    out_layer = tf.add(out_layer, biases['disc2_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , data.shape[0])
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i].toarray() for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    data_shuffle = scipy.sparse.csr_matrix(data_shuffle)
    print(data_shuffle.shape)
    return data_shuffle, labels_shuffle
#     print(len(data_shuffle[0]))
#     return scipy.sparse.csr_matrix(data_shuffle), np.asarray(labels_shuffle)
#     return np.asarray(data_shuffle), np.asarray(labels_shuffle)

# https://stackoverflow.com/questions/40896157/scipy-sparse-csr-matrix-to-tensorflow-sparsetensor-mini-batch-gradient-descent
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

121394
Instructions for updating:
Colocations handled automatically by placer.


In [10]:
# Build Networks
# Network Inputs
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
# disc_input_normal = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_normal')
# disc_input_real_susp = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_real_susp')
# # disc_input_fake_susp = tf.placeholder(tf.float32, shape=[None, image_dim], name='disc_input_fake_susp')

# tf.float32
# https://towardsdatascience.com/how-to-use-dataset-in-tensorflow-c758ef9e4428
# x_norm = tf.sparse_placeholder(tf.float64)
# x_susp = tf.sparse_placeholder(tf.float64)
x_norm, x_susp = tf.placeholder(tf.float32, shape=[None, image_dim]), tf.placeholder(tf.float32, shape=[None, image_dim])
dataset_norm = tf.data.Dataset.from_tensor_slices(x_norm).repeat().batch(batch_size)
dataset_susp = tf.data.Dataset.from_tensor_slices(x_susp).repeat().batch(batch_size)
# train_data = (np.random.sample((100,2)), np.random.sample((100,1)))
# test_data = (np.array([[1,2]]), np.array([[0]]))
iter_norm = dataset_norm.make_initializable_iterator()
iter_susp = dataset_susp.make_initializable_iterator()
features_norm = iter_norm.get_next()
features_susp = iter_susp.get_next()

# Build Generator Network
# gen_sample_pre = generator(features_susp)
gen_sample_pre = generator(gen_input)
gen_sample = generator(gen_input)

# Build 2 Discriminator Networks (one from noise input, one from generated samples)
# disc_SCI_normal = discriminator_SCI(disc_input_normal)
# disc_SCI_susp_real = discriminator_SCI(disc_input_real_susp)
# disc_SCI_susp_fake = discriminator_SCI(gen_sample)
# disc_gvr_real = discriminator_gvr(disc_input_real_susp)
# disc_gvr_fake = discriminator_gvr(gen_sample)

disc_SCI_normal = discriminator_SCI(features_norm)
disc_SCI_susp_real = discriminator_SCI(features_susp)
disc_SCI_susp_fake = discriminator_SCI(gen_sample)
disc_gvr_real = discriminator_gvr(features_susp)
disc_gvr_fake = discriminator_gvr(gen_sample)


# Build Loss
gen_loss_pre = tf.losses.mean_squared_error(features_susp, gen_sample_pre)
gen_loss = -tf.reduce_mean(tf.log(disc_SCI_susp_fake + 1e-8) + tf.log(disc_gvr_fake + 1e-8))
# disc_SCI_loss = -tf.reduce_mean(tf.log(disc_SCI_normal + 1e-8) + tf.log(1. - disc_SCI_susp_real + 1e-8) + tf.log(1. - disc_SCI_susp_fake + 1e-8))
disc_SCI_loss = -tf.reduce_mean(tf.log(disc_SCI_normal + 1e-8) + tf.log(1. - disc_SCI_susp_real + 1e-8))
disc_gvr_loss = -tf.reduce_mean(tf.log(disc_gvr_real + 1e-8) + tf.log(1. - disc_gvr_fake + 1e-8))

# Build Optimizers
optimizer_gen_pre = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Training Variables for each optimizer
# By default in TensorFlow, all variables are updated by each optimizer, so we
# need to precise for each one of them the specific variables to update.
# Generator Network Variables
gen_vars = [weights['gen_hidden1'], weights['gen_out'],
            biases['gen_hidden1'], biases['gen_out']]
# Discriminator Network Variables
disc_SCI_vars = [weights['disc1_hidden1'], weights['disc1_out'],
            biases['disc1_hidden1'], biases['disc1_out']]
disc_gvr_vars = [weights['disc2_hidden1'], weights['disc2_out'],
            biases['disc2_hidden1'], biases['disc2_out']]

# Create training operations
train_gen_pre = optimizer_gen_pre.minimize(gen_loss_pre, var_list=gen_vars)
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_disc_SCI = optimizer_disc.minimize(disc_SCI_loss, var_list=disc_SCI_vars)
train_disc_gvr = optimizer_disc.minimize(disc_gvr_loss, var_list=disc_gvr_vars)

print(X_train_norm.shape)
print(X_train_susp.shape)

(13802, 121394)
(901, 121394)


In [11]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    sess.run([iter_norm.initializer, iter_susp.initializer], feed_dict={ x_norm: X_train_norm.todense(), x_susp: X_train_susp.todense()})
    
    print('Pretraining of generator starting.')
    for i in range(1, num_steps*100+1):
        z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
        # Train
#         feed_dict = {disc_input: batch_x, gen_input: z}
        feed_dict = {gen_input: z}
        _, glpre = sess.run([train_gen_pre, gen_loss_pre], feed_dict=feed_dict)
        print('Step %i: Generator Loss: %f' % (i, glpre))
    
#     sess.run([iter_norm.initializer, iter_susp.initializer], feed_dict={ x_norm: X_train_norm.todense(), x_susp: X_train_susp.todense()})
    print('Starting actual training')
    for i in range(1, num_steps+1):
        # Prepare Data
        # Get the next batch of MNIST data (only images are needed, not labels)
#         batch_x, _ = mnist.train.next_batch(batch_size)
#         batch_x_norm, _ = next_batch(batch_size, X_train_norm, y_train_norm)
#         batch_x_susp, _ = next_batch(batch_size, X_train_susp, y_train_susp)
#         print(batch_x_norm.shape)
#         print(batch_x_susp.shape)
        # Generate noise to feed to the generator
        z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
        # Train
#         feed_dict = {disc_input: batch_x, gen_input: z}
        feed_dict = {gen_input: z}
        _, _, _, gl, dSCIl, dgvrl = sess.run([train_gen, train_disc_SCI, train_disc_gvr, gen_loss, disc_SCI_loss, disc_gvr_loss],
                                feed_dict=feed_dict)
#         if i % 1000 == 0 or i == 1:
        print('Step %i: Generator Loss: %f, Disc SCI Loss: %f, Disc gvr Loss: %f' % (i, gl, dSCIl, dgvrl))

    print("Finished Training")
    # test SCI
    sess.run(iter_norm.initializer, feed_dict={ x_norm: X_test.todense()})
    y_pred = sess.run(disc_SCI_normal)
    print(metrics.accuracy_score(y_test, y_pred))
#     # Generate images from noise, using the generator network.
#     f, a = plt.subplots(4, 10, figsize=(10, 4))
#     for i in range(10):
#         # Noise input.
#         z = np.random.uniform(-1., 1., size=[4, noise_dim])
#         g = sess.run([gen_sample], feed_dict={gen_input: z})
#         g = np.reshape(g, newshape=(4, 28, 28, 1))
#         # Reverse colours for better display
#         g = -1 * (g - 1)
#         for j in range(4):
#             # Generate image from noise. Extend to 3 channels for matplot figure.
#             img = np.reshape(np.repeat(g[j][:, :, np.newaxis], 3, axis=2),
#                              newshape=(28, 28, 3))
#             a[j][i].imshow(img)

#     f.show()
#     plt.draw()
#     plt.waitforbuttonpress()

Pretraining of generator starting.
Step 1: Generator Loss: 0.279664
Step 2: Generator Loss: 0.280260
Step 3: Generator Loss: 0.280477
Step 4: Generator Loss: 0.283691
Step 5: Generator Loss: 0.279317
Step 6: Generator Loss: 0.280007
Step 7: Generator Loss: 0.279862
Step 8: Generator Loss: 0.281284
Step 9: Generator Loss: 0.277914
Step 10: Generator Loss: 0.278026
Step 11: Generator Loss: 0.279731
Step 12: Generator Loss: 0.276324
Step 13: Generator Loss: 0.280012
Step 14: Generator Loss: 0.280907
Step 15: Generator Loss: 0.277576
Step 16: Generator Loss: 0.278840
Step 17: Generator Loss: 0.277607
Step 18: Generator Loss: 0.277580
Step 19: Generator Loss: 0.278943
Step 20: Generator Loss: 0.277178
Step 21: Generator Loss: 0.278500
Step 22: Generator Loss: 0.279537
Step 23: Generator Loss: 0.277930
Step 24: Generator Loss: 0.276545
Step 25: Generator Loss: 0.275696
Step 26: Generator Loss: 0.276830
Step 27: Generator Loss: 0.275138
Step 28: Generator Loss: 0.276137
Step 29: Generator Los

Step 238: Generator Loss: 0.242442
Step 239: Generator Loss: 0.242606
Step 240: Generator Loss: 0.242597
Step 241: Generator Loss: 0.241100
Step 242: Generator Loss: 0.241418
Step 243: Generator Loss: 0.241706
Step 244: Generator Loss: 0.241767
Step 245: Generator Loss: 0.240793
Step 246: Generator Loss: 0.240348
Step 247: Generator Loss: 0.239176
Step 248: Generator Loss: 0.240634
Step 249: Generator Loss: 0.240717
Step 250: Generator Loss: 0.240145
Step 251: Generator Loss: 0.242668
Step 252: Generator Loss: 0.240081
Step 253: Generator Loss: 0.238880
Step 254: Generator Loss: 0.240689
Step 255: Generator Loss: 0.239240
Step 256: Generator Loss: 0.239473
Step 257: Generator Loss: 0.240209
Step 258: Generator Loss: 0.239293
Step 259: Generator Loss: 0.239114
Step 260: Generator Loss: 0.240237
Step 261: Generator Loss: 0.239324
Step 262: Generator Loss: 0.238740
Step 263: Generator Loss: 0.241301
Step 264: Generator Loss: 0.238450
Step 265: Generator Loss: 0.238225
Step 266: Generator 

Step 473: Generator Loss: 0.207870
Step 474: Generator Loss: 0.207713
Step 475: Generator Loss: 0.208481
Step 476: Generator Loss: 0.208279
Step 477: Generator Loss: 0.207608
Step 478: Generator Loss: 0.207820
Step 479: Generator Loss: 0.207127
Step 480: Generator Loss: 0.207389
Step 481: Generator Loss: 0.207869
Step 482: Generator Loss: 0.207069
Step 483: Generator Loss: 0.207056
Step 484: Generator Loss: 0.207222
Step 485: Generator Loss: 0.207881
Step 486: Generator Loss: 0.204656
Step 487: Generator Loss: 0.207301
Step 488: Generator Loss: 0.207172
Step 489: Generator Loss: 0.206176
Step 490: Generator Loss: 0.206938
Step 491: Generator Loss: 0.205260
Step 492: Generator Loss: 0.207981
Step 493: Generator Loss: 0.203722
Step 494: Generator Loss: 0.205728
Step 495: Generator Loss: 0.206081
Step 496: Generator Loss: 0.205160
Step 497: Generator Loss: 0.206971
Step 498: Generator Loss: 0.205942
Step 499: Generator Loss: 0.205516
Step 500: Generator Loss: 0.204130
Step 501: Generator 

Step 708: Generator Loss: 0.169049
Step 709: Generator Loss: 0.172737
Step 710: Generator Loss: 0.172504
Step 711: Generator Loss: 0.172961
Step 712: Generator Loss: 0.171428
Step 713: Generator Loss: 0.169616
Step 714: Generator Loss: 0.173018
Step 715: Generator Loss: 0.174008
Step 716: Generator Loss: 0.171600
Step 717: Generator Loss: 0.168977
Step 718: Generator Loss: 0.169319
Step 719: Generator Loss: 0.170400
Step 720: Generator Loss: 0.170590
Step 721: Generator Loss: 0.170426
Step 722: Generator Loss: 0.174038
Step 723: Generator Loss: 0.171086
Step 724: Generator Loss: 0.173796
Step 725: Generator Loss: 0.169265
Step 726: Generator Loss: 0.168704
Step 727: Generator Loss: 0.168145
Step 728: Generator Loss: 0.168954
Step 729: Generator Loss: 0.175170
Step 730: Generator Loss: 0.169343
Step 731: Generator Loss: 0.168247
Step 732: Generator Loss: 0.168499
Step 733: Generator Loss: 0.170410
Step 734: Generator Loss: 0.167339
Step 735: Generator Loss: 0.170187
Step 736: Generator 

Step 943: Generator Loss: 0.136243
Step 944: Generator Loss: 0.138281
Step 945: Generator Loss: 0.136646
Step 946: Generator Loss: 0.133406
Step 947: Generator Loss: 0.133212
Step 948: Generator Loss: 0.136317
Step 949: Generator Loss: 0.136267
Step 950: Generator Loss: 0.133938
Step 951: Generator Loss: 0.132983
Step 952: Generator Loss: 0.136854
Step 953: Generator Loss: 0.134116
Step 954: Generator Loss: 0.134556
Step 955: Generator Loss: 0.133684
Step 956: Generator Loss: 0.132151
Step 957: Generator Loss: 0.135623
Step 958: Generator Loss: 0.132383
Step 959: Generator Loss: 0.138864
Step 960: Generator Loss: 0.138889
Step 961: Generator Loss: 0.135500
Step 962: Generator Loss: 0.133312
Step 963: Generator Loss: 0.131077
Step 964: Generator Loss: 0.137684
Step 965: Generator Loss: 0.138052
Step 966: Generator Loss: 0.134252
Step 967: Generator Loss: 0.139598
Step 968: Generator Loss: 0.135644
Step 969: Generator Loss: 0.134481
Step 970: Generator Loss: 0.131606
Step 971: Generator 

MemoryError: 

So we are getting a diverging GAN error, according to https://github.com/soumith/ganhacks/issues/14, this may be because

Pretraining the generator with real susp conversations is not working since in training, noise is used as generator's input